In [205]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from math import pi

from bokeh.io import show, output_file
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [206]:
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
prod_ing = prod_ing.sort_values(['id', 'order'])

print('Number of products: ', len(prods))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products:  15945
Number of unique ingredients:  7325
Number of total ingredients:  545617


### Ings
Ings df is just a database of unique ingredients and their harmful score, function and notes.

In [207]:
#caught a rogue ingredient
ings['ingredient'] = ings['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')
ings = ings.drop_duplicates(subset = 'ingredient')
print('Number of unique ingredients: ', len(ings))
ings.head()

Number of unique ingredients:  7324


,id,ingredient,ewg,cir,func_Abrasive,func_Antimicrobial,func_Antioxidant,func_Antistatic Agent,func_Astringent,func_Binding,...,notes_Comedogenic Rating (3),notes_Comedogenic Rating (4),notes_Comedogenic Rating (5),notes_Good for Dry Skin,notes_Good for Oily Skin,notes_Good for Sensitive Skin,notes_Paraben,notes_Promotes Wound Healing,notes_Sulfate,notes_UV Protection
0,0,Water,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Hydrogenated Polyisobutene,1,A,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Butylene Glycol,1,A,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,Sesamum Indicum (Sesame) Seed Oil,1,A,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,Ammonium Acryloyldimethyltaurate/VP Copolymer,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [208]:
ing_uniqueID = ings.loc[:,['ingredient']].reset_index()
ing_uniqueID = ing_uniqueID.drop(['index'], axis = 1)
ing_uniqueID['uniqueID'] = ing_uniqueID.index
#ing_uniqueID

### Prods
Prods df is a database of all the products with various information including brand, price and rating

In [209]:
prods.head(3)

,id,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,cat_Cleansers,...,note_AlcoholFree,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection
0,0,Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
2,2,Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,0,0,...,1,0,1,0,1,0,0,1,1,0
3,3,Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,0,0,...,0,0,1,1,0,0,1,1,1,0


### Prod_ing
Prod ing is a database that essentiall links prods and ings. it contains the product id, ingredient and ingredient order.

In [210]:
prod_ing.head(3)

,id,ingredient,order
0,0,Water,1
1,0,Hydrogenated Polyisobutene,2
2,0,Butylene Glycol,3


### Add a column that is a unique ingredient identifier

In [211]:
#id is product id
#uniqueID is ingredient ID
prod_ing = pd.merge(prod_ing, ing_uniqueID, on='ingredient')
prod_ing = prod_ing.sort_values(['id', 'order'])
prod_ing.head()

,id,ingredient,order,uniqueID
0,0,Water,1,0
15954,0,Hydrogenated Polyisobutene,2,1
16617,0,Butylene Glycol,3,2
24563,0,Sesamum Indicum (Sesame) Seed Oil,4,3
24889,0,Ammonium Acryloyldimethyltaurate/VP Copolymer,5,4


In [213]:
#reset index and drop
prod_ing = prod_ing.reset_index(drop=True)

## Reformatting Prod_ing
Instead of having each ingredient in it's own row, we want to create a list of ingredients for each product while retaining the order.

In [215]:
prod_ing.groupby('id')['ingredient'].apply(list)

id
0        [Water, Hydrogenated Polyisobutene, Butylene G...
1        [Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...
2        [Water, Glycerin, Cetearyl Alcohol, Petrolatum...
3        [Water, Glycereth-26, Alcohol, Butylene Glycol...
4        [Petrolatum, Glycerin, Cocamidopropyl Betaine,...
5        [Water, Butylene Glycol, Glycerin, Disodium Su...
6        [Mineral Oil, Avena Sativa (Oat) Kernel Flour,...
7        [Sodium Palm Kernelate, Water, Sodium Borate, ...
8        [Water, Hamamelis Virginiana (Witch Hazel) Wat...
9        [Water, Caprylic/Capric Triglyceride, Glycerin...
10       [Water, Glycerin, Sodium Laureth Sulfate, PEG-...
11       [Water, Tri-C14-15 Alkyl Citrate, Cyclohexasil...
12       [Water, Butylene Glycol, Propylene Glycol, Dip...
13       [Sodium Palm Kernelate, Water, Petrolatum, Gly...
14       [Water, Chamaecyparis Obtusa Water, Glycerin, ...
15       [Snail Secretion Filtrate, Water, Butylene Gly...
16       [Water, Bentonite, Glycerin, Magnesium Alumi

In [216]:
#cleanup of a rogue ingredient here as well
prod_ing['ingredient'] = prod_ing['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')

##group and create list
prod_ing_lists = prod_ing.groupby('id')['ingredient'].apply(list)
prod_ing_ID_lists = prod_ing.groupby('id')['uniqueID'].apply(list)

##convert back to dataframe and reset index
prod_ing_df = prod_ing_lists.to_frame().reset_index()
prod_ing_ID_df = prod_ing_ID_lists.to_frame().reset_index()
##check to make sure unique id's are still in tact (id shouldn't be exactly == to index)
prod_ing_lists.head()

id
0    [Water, Hydrogenated Polyisobutene, Butylene G...
1    [Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...
2    [Water, Glycerin, Cetearyl Alcohol, Petrolatum...
3    [Water, Glycereth-26, Alcohol, Butylene Glycol...
4    [Petrolatum, Glycerin, Cocamidopropyl Betaine,...
Name: ingredient, dtype: object

In [217]:
prod_ing_df.tail()

,id,ingredient
19858,19922,"[Water, Dimethicone, Glycerin, Palmitic Acid(E..."
19859,19923,"[Water, Dipropylene Glycol, Caprylic/Capric Tr..."
19860,19924,"[Water, Polyvinyl Alcohol, Ethanolamine, 1,2-H..."
19861,19925,"[Ricinus Communis (Castor) Seed Oil, Lanolin, ..."
19862,19926,"[Water, Ascorbic Acid (Vitamin C, Glycerin, Di..."


In [218]:
prod_ing_lists = pd.merge(prod_ing_df, prod_ing_ID_df, on='id')
prod_ing_lists = prod_ing_lists.rename(columns = {'ingredient': 'ingList', 'uniqueID': 'ing#List' })
prod_ing_lists.head()

,id,ingList,ing#List
0,0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,1,"[Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 3..."
2,2,"[Water, Glycerin, Cetearyl Alcohol, Petrolatum...","[0, 23, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55..."
3,3,"[Water, Glycereth-26, Alcohol, Butylene Glycol...","[0, 62, 63, 2, 64, 65, 66, 67, 68, 69, 70, 71,..."
4,4,"[Petrolatum, Glycerin, Cocamidopropyl Betaine,...","[47, 23, 92, 93, 94, 2, 95, 96, 97, 98, 99, 16..."


## Merge prod_ings and prods
We can now use the list to merge back with the products.

In [219]:
products_and_ingredients= pd.merge(prod_ing_lists, prods, on='id')
products_and_ingredients.tail(3)
len(products_and_ingredients)

15945

### Add column for number of ingredients

In [220]:
products_and_ingredients['ingCount'] = products_and_ingredients['ingList'].apply(lambda x: len(x))
products_and_ingredients.head(3)

,id,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,...,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection,ingCount
0,0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,...,0,0,0,1,0,0,0,1,0,22
1,2,"[Water, Glycerin, Cetearyl Alcohol, Petrolatum...","[0, 23, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55...",Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,...,0,1,0,1,0,0,1,1,0,20
2,3,"[Water, Glycereth-26, Alcohol, Butylene Glycol...","[0, 62, 63, 2, 64, 65, 66, 67, 68, 69, 70, 71,...",Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,...,0,1,1,0,0,1,1,1,0,38


In [222]:
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].apply(lambda x: str(x))
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace(',', '')
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace('[', '')
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace(']', '')
products_and_ingredients = products_and_ingredients.drop_duplicates(subset = 'product')

products_and_ingredients.head()
len(products_and_ingredients)

15945

# Recommendation engine
Based on the one detailed [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

In [223]:
#Import TfIdfVectorizer from scikit-learn
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object
tfidf = TfidfVectorizer()
tfidf = TfidfVectorizer(stop_words=['Water'])

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(products_and_ingredients['ing#List'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(15945, 6888)

In [269]:
tfidf_matrix

<15945x6888 sparse matrix of type '<class 'numpy.float64'>'
	with 421665 stored elements in Compressed Sparse Row format>

#### Create a cosine similarity matrix 

In [263]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(15945, 15945)

In [265]:
#Creaet an index of all products
indices = pd.Series(products_and_ingredients.index, index=products_and_ingredients['product']).drop_duplicates()

In [227]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(product, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[product]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return products_and_ingredients[['product', 'ingList']].iloc[product_indices]

In [228]:
get_recommendations('Foot Cream Norwegian Formula')

,product,ingList
5406,Daily Serum,"[Water, Butylene Glycol, Glycerin, Propylene G..."
5726,Environmental Rescue Mask,"[Water, Butylene Glycol, Glycerin, Propylene G..."
7770,Gesichtswasser Soft & Clear,"[Water, Alcohol Denat., Glycerin, Polyglyceryl..."
14288,Private Formula Day Cream Moisturiser,"[Water, Dimethicone, Prunus Armeniaca (Apricot..."
9356,Ultra-Gentle Facial Moisturizer (Oil Free),"[Water, Glycerin, Ethylhexyl Palmitate, Dimeth..."
1142,Moisture Oil-Free for Sensitive Skin (Canadian...,"[Water, Glycerin, Ethylhexyl Palmitate, Dimeth..."
1148,"Oil-Free Moisture, for Sensitive Skin","[Water, Glycerin, Ethylhexyl Palmitate, Dimeth..."
7352,Oil-Free Moisturizing Facial Lotion,"[Water, Glycerin, Ethylhexyl Palmitate, Dimeth..."
4966,Dual Treatment Moisture Lotion with SPF 15,"[Cinoxate, Ethylhexyl Salicylate, Water, Octyl..."
1227,Oil-Free Moisture Sensitive Skin Ultra-Gentle ...,"[Water, Glycerin, Ethylhexyl Palmitate, Dimeth..."


In [229]:
ingTest = products_and_ingredients[products_and_ingredients['product'].str.match('Foot Cream Norwegian Formula')]
ingTest = ingTest['ingList']
print(*ingTest, sep='\n')

['Water', 'Glycerin', 'Cetearyl Alcohol', 'Petrolatum', 'Cyclomethicone', 'Panthenol', 'Dimethicone', 'Keratin Amino Acids', 'Menthol', 'Allantoin', 'Dilauryl Thiodipropionate', 'Alpha-Bisabolol', 'Tocopheryl Linoleate', 'Sodium Cetearyl Sulfate', 'Sodium Sulfate', 'Methylparaben', 'Propylparaben', 'Diazolidinyl Urea', 'Sodium Chloride', 'Fragrance']


### RBO similarity measure

Based on code found [here](https://github.com/dlukes/rbo/blob/master/rbo.py)

In [272]:
#Comute the rbo similarity matrix
len(products_and_ingredients)

15945

In [231]:
def rbo(list1, list2, p):
    """Complete RBO analysis (lower bound, residual, point estimate).
    ``list`` arguments should be already correctly sorted iterables and each
    item should either be an atomic value or a set of values tied for that
    rank. ``p`` is the probability of looking for overlap at rank k + 1 after
    having examined rank k.
    """
    if not 0 <= p <= 1:
        raise ValueError("The ``p`` parameter must be between 0 and 1.")
    args = (list1, list2, p)
    return dict(min=rbo_min(*args), res=rbo_res(*args), ext=rbo_ext(*args))

In [337]:
list1 = products_and_ingredients[products_and_ingredients['product'].str.match('Foot Cream Norwegian Formula')]
list1 = list(list1['ingList'])
list1 = list1[0]

list2 = products_and_ingredients[products_and_ingredients['product'].str.match('Dual Treatment Moisture Lotion with SPF 15')]
list2 = list(list2['ingList'])
list2 = list2[0]


In [341]:
average_overlap(list1, list2)

1.0

In [320]:
products_and_ingredients.iloc[10,1]

['Sodium Palm Kernelate',
 'Water',
 'Petrolatum',
 'Glycerin',
 'Sodium Chloride',
 'Trisodium HEDTA',
 'Titanium Dioxide',
 'Iron Oxides',
 'Yellow 5 Lake']

In [340]:
list1 = products_and_ingredients.iloc[0,1]
list2 = products_and_ingredients.iloc[0,1]

In [342]:
items = len(products_and_ingredients) ##change to len(products_ing_list) to generate full dataframe
AO_sim = np.zeros((items,items))
AO_sim = pd.DataFrame(AO_sim)

for i in range(0, items):
    list1 = products_and_ingredients.iloc[i,1]
    print("column: ", i)
    for j in range(0, items):
        if j == i:
            rbo_ext_sim.iloc[i,j] = 1.00
        else:
            list2 = products_and_ingredients.iloc[j,1]
            AO = average_overlap(list1, list2)
            AO_sim.iloc[i,j] = AO

column:  0
column:  1
column:  2
column:  3
column:  4
column:  5
column:  6
column:  7
column:  8
column:  9
column:  10
column:  11
column:  12
column:  13
column:  14
column:  15
column:  16
column:  17
column:  18
column:  19
column:  20
column:  21
column:  22
column:  23
column:  24
column:  25
column:  26
column:  27
column:  28
column:  29
column:  30
column:  31
column:  32
column:  33
column:  34
column:  35
column:  36
column:  37
column:  38
column:  39
column:  40
column:  41
column:  42
column:  43
column:  44
column:  45
column:  46
column:  47
column:  48
column:  49
column:  50
column:  51
column:  52
column:  53
column:  54
column:  55
column:  56
column:  57
column:  58
column:  59
column:  60
column:  61
column:  62
column:  63
column:  64
column:  65
column:  66
column:  67
column:  68
column:  69
column:  70
column:  71
column:  72
column:  73
column:  74
column:  75
column:  76
column:  77
column:  78
column:  79
column:  80
column:  81
column:  82
column:  83
co

KeyboardInterrupt: 

In [307]:
rbo(list1, list2, 0.9)

{'ext': 0.2923571478196387,
 'min': 0.26541744409298035,
 'res': 0.07678869216762815}

In [335]:
average_overlap(list1, list2)

0.275634137143777

In [332]:
#!/usr/bin/env python3

"""Rank-biased overlap, a ragged sorted list similarity measure.
See http://doi.acm.org/10.1145/1852102.1852106 for details. All functions
directly taken from the paper are named so that they can be clearly
cross-identified.
The definition of overlap has been modified to account for ties. Without this,
results for lists with tied items were being inflated. The modification itself
is not mentioned in the paper but seems to be reasonable, see function
``overlap()``. Places in the code which diverge from the spec in the paper
because of this are highlighted with comments.
"""

import math
from bisect import bisect_left


def _numtest(floatn):
    return "{:.3f}".format(floatn)


def set_at_depth(lst, depth):
    ans = set()
    for v in lst[:depth]:
        if isinstance(v, set):
            ans.update(v)
        else:
            ans.add(v)
    return ans


def raw_overlap(list1, list2, depth):
    """Overlap as defined in the article.
    """
    set1, set2 = set_at_depth(list1, depth), set_at_depth(list2, depth)
    return len(set1.intersection(set2)), len(set1), len(set2)


def overlap(list1, list2, depth):
    """Overlap which accounts for possible ties.
    This isn't mentioned in the paper but should be used in the ``rbo*()``
    functions below, otherwise overlap at a given depth might be > depth which
    inflates the result.
    There are no guidelines in the paper as to what's a good way to calculate
    this, but a good guess is agreement scaled by the minimum between the
    requested depth and the lengths of the considered lists (overlap shouldn't
    be larger than the number of ranks in the shorter list, otherwise results
    are conspicuously wrong when the lists are of unequal lengths -- rbo_ext is
    not between rbo_min and rbo_min + rbo_res.
    >>> overlap("abcd", "abcd", 3)
    3.0
    >>> overlap("abcd", "abcd", 5)
    4.0
    >>> overlap(["a", {"b", "c"}, "d"], ["a", {"b", "c"}, "d"], 2)
    2.0
    >>> overlap(["a", {"b", "c"}, "d"], ["a", {"b", "c"}, "d"], 3)
    3.0
    """
    return agreement(list1, list2, depth) * min(depth, len(list1), len(list2))
    # NOTE: comment the preceding and uncomment the following line if you want
    # to stick to the algorithm as defined by the paper
    # return raw_overlap(list1, list2, depth)[0]


def agreement(list1, list2, depth):
    """Proportion of shared values between two sorted lists at given depth.
    >>> _numtest(agreement("abcde", "abdcf", 1))
    '1.000'
    >>> _numtest(agreement("abcde", "abdcf", 3))
    '0.667'
    >>> _numtest(agreement("abcde", "abdcf", 4))
    '1.000'
    >>> _numtest(agreement("abcde", "abdcf", 5))
    '0.800'
    >>> _numtest(agreement([{1, 2}, 3], [1, {2, 3}], 1))
    '0.667'
    >>> _numtest(agreement([{1, 2}, 3], [1, {2, 3}], 2))
    '1.000'
    """
    len_intersection, len_set1, len_set2 = raw_overlap(list1, list2, depth)
    return 2 * len_intersection / (len_set1 + len_set2)


def cumulative_agreement(list1, list2, depth):
    return (agreement(list1, list2, d) for d in range(1, depth + 1))


def average_overlap(list1, list2, depth=None):
    """Calculate average overlap between ``list1`` and ``list2``.
    >>> _numtest(average_overlap("abcdefg", "zcavwxy", 1))
    '0.000'
    >>> _numtest(average_overlap("abcdefg", "zcavwxy", 2))
    '0.000'
    >>> _numtest(average_overlap("abcdefg", "zcavwxy", 3))
    '0.222'
    >>> _numtest(average_overlap("abcdefg", "zcavwxy", 4))
    '0.292'
    >>> _numtest(average_overlap("abcdefg", "zcavwxy", 5))
    '0.313'
    >>> _numtest(average_overlap("abcdefg", "zcavwxy", 6))
    '0.317'
    >>> _numtest(average_overlap("abcdefg", "zcavwxy", 7))
    '0.312'
    """
    depth = min(len(list1), len(list2)) if depth is None else depth
    return sum(cumulative_agreement(list1, list2, depth)) / depth


def rbo_at_k(list1, list2, p, depth=None):
    # ``p**d`` here instead of ``p**(d - 1)`` because enumerate starts at
    # 0
    depth = min(len(list1), len(list2)) if depth is None else depth
    d_a = enumerate(cumulative_agreement(list1, list2, depth))
    return (1 - p) * sum(p**d * a for (d, a) in d_a)


def rbo_min(list1, list2, p, depth=None):
    """Tight lower bound on RBO.
    See equation (11) in paper.
    >>> _numtest(rbo_min("abcdefg", "abcdefg", .9))
    '0.767'
    >>> _numtest(rbo_min("abcdefgh", "abcdefg", .9))
    '0.767'
    """
    depth = min(len(list1), len(list2)) if depth is None else depth
    x_k = overlap(list1, list2, depth)
    log_term = x_k * math.log(1 - p)
    sum_term = sum(p**d / d * (overlap(list1, list2, d) - x_k)
                   for d in range(1, depth + 1))
    return (1 - p) / p * (sum_term - log_term)


def rbo_res(list1, list2, p):
    """Upper bound on residual overlap beyond evaluated depth.
    See equation (30) in paper.
    NOTE: The doctests weren't verified but seem plausible. In particular, for
    identical lists, ``rbo_min()`` and ``rbo_res()`` should add up to 1, which
    is the case.
    >>> _numtest(rbo_res("abcdefg", "abcdefg", .9))
    '0.233'
    >>> _numtest(rbo_res("abcdefg", "abcdefghijklmnopqrstuvwxyz", .9))
    '0.239'
    """
    S, L = sorted((list1, list2), key=len)
    s, l = len(S), len(L)
    x_l = overlap(list1, list2, l)
    # since overlap(...) can be fractional in the general case of ties and f
    # must be an integer → math.ceil()
    f = math.ceil(l + s - x_l)
    # upper bound of range() is non-inclusive, therefore + 1 is needed
    term1 = s * sum(p**d / d for d in range(s + 1, f + 1))
    term2 = l * sum(p**d / d for d in range(l + 1, f + 1))
    term3 = x_l * (math.log(1 / (1 - p)) - sum(p**d / d for d in range(1, f + 1)))
    return p**s + p**l - p**f - (1 - p) / p * (term1 + term2 + term3)


def rbo_ext(list1, list2, p):
    """RBO point estimate based on extrapolating observed overlap.
    See equation (32) in paper.
    NOTE: The doctests weren't verified but seem plausible.
    >>> _numtest(rbo_ext("abcdefg", "abcdefg", .9))
    '1.000'
    >>> _numtest(rbo_ext("abcdefg", "bacdefg", .9))
    '0.900'
    """
    S, L = sorted((list1, list2), key=len)
    s, l = len(S), len(L)
    x_l = overlap(list1, list2, l)
    x_s = overlap(list1, list2, s)
    # the paper says overlap(..., d) / d, but it should be replaced by
    # agreement(..., d) defined as per equation (28) so that ties are handled
    # properly (otherwise values > 1 will be returned)
    # sum1 = sum(p**d * overlap(list1, list2, d)[0] / d for d in range(1, l + 1))
    sum1 = sum(p**d * agreement(list1, list2, d) for d in range(1, l + 1))
    sum2 = sum(p**d * x_s * (d - s) / s / d for d in range(s + 1, l + 1))
    term1 = (1 - p) / p * (sum1 + sum2)
    term2 = p**l * ((x_l - x_s) / l + x_s / s)
    return term1 + term2


def rbo(list1, list2, p):
    """Complete RBO analysis (lower bound, residual, point estimate).
    ``list`` arguments should be already correctly sorted iterables and each
    item should either be an atomic value or a set of values tied for that
    rank. ``p`` is the probability of looking for overlap at rank k + 1 after
    having examined rank k.
    """
    if not 0 <= p <= 1:
        raise ValueError("The ``p`` parameter must be between 0 and 1.")
    args = (list1, list2, p)
    return dict(min=rbo_min(*args), res=rbo_res(*args), ext=rbo_ext(*args))


def sort_dict(dct):
    scores = []
    items = []
    # items should be unique, scores don't have to
    for item, score in dct.items():
        # sort in descending order, i.e. according to ``-score``
        score = -score
        i = bisect_left(scores, score)
        if i == len(scores):
            scores.append(score)
            items.append(item)
        elif scores[i] == score:
            existing_item = items[i]
            if isinstance(existing_item, set):
                existing_item.add(item)
            else:
                items[i] = {existing_item, item}
        else:
            scores.insert(i, score)
            items.insert(i, item)
    return items


def rbo_dict(dict1, dict2, p):
    """Wrapper around ``rbo()`` for dict input.
    Each dict maps items to be sorted to the score according to which they
    should be sorted.
    """
    list1, list2 = sort_dict(dict1), sort_dict(dict2)
    return rbo(list1, list2, p)


if __name__ in ("__main__", "__console__"):
    import doctest
    doctest.testmod()